 <h1>HR AI Agent</h1>

<h2>Build an AI agent that autonomously screens job applicants from a pool of resumes,
ranks candidates, schedules interviews</h2>

In [1]:
import warnings

In [2]:
pip install PyPDF2 python-docx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import openai
openai.api_key = "sk-your-real-key-here"


In [5]:
import os
import PyPDF2
import docx
import json
import openai

# 1. Resume text extraction
def extract_text_from_resume(file_path):
    _, file_extension = os.path.splitext(file_path)
    file_extension = file_extension.lower()

    if file_extension == '.pdf':
        try:
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = ''.join([page.extract_text() or '' for page in reader.pages])
                return text
        except Exception as e:
            print(f"Error reading PDF: {e}")
            return None

    elif file_extension == '.docx':
        try:
            doc = docx.Document(file_path)
            return ' '.join([para.text for para in doc.paragraphs])
        except Exception as e:
            print(f"Error reading DOCX: {e}")
            return None

    else:
        print(f"Unsupported format: {file_extension}")
        return None


# 2. LLM ranking & summarization
def rank_and_summarize_candidate(resume_text, job_description):
    prompt = f"""
    You are an expert HR AI assistant. 

    Job Description:
    {job_description}

    Candidate Resume:
    {resume_text}

    Provide output as JSON:
    {{
      "score": (integer from 1-100),
      "summary": "string summary"
    }}
    """

    try:
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        
        content = response.choices[0].message["content"]
        return json.loads(content)

    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return None


# Example Usage
if __name__ == "__main__":
    job_description_text = "Looking for a Data Analyst with Python, SQL, and Power BI experience."
    resume_text = extract_text_from_resume("BISMA RESUME2025.pdf")  
    
    if resume_text:
        candidate_info = rank_and_summarize_candidate(resume_text, job_description_text)
        if candidate_info:
            print(f"Candidate Score: {candidate_info['score']}")
            print(f"Candidate Summary: {candidate_info['summary']}")


Error calling OpenAI API: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-your-*********here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
